# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [30]:
# Initialize and constants

OLLAMA_HOST = 'http://localhost:11434/v1'
MODEL = 'llama3.2:1b'

ollama = OpenAI(base_url=OLLAMA_HOST, api_key='ollama')

In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\n\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")

In [5]:
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [49]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, Pricing page, or Careers/Jobs pages. Type have to be decided from link not vice versa\
verify that link format in response is correct"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [45]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links, in this case transform them into full form):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [39]:
def get_links(url):
    website = Website(url)
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [14]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 'inference/get-started',
 '/spaces',
 '/models',
 '/openai/gpt-oss-120b',
 '/openai/gpt-oss-20b',
 '/Qwen/Qwen-Image',
 '/tencent/Hunyuan-1.8B-Instruct',
 '/black-forest-labs/FLUX.1-Krea-dev',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Qwen/Qwen-Image',
 '/spaces/black-forest-labs/FLUX.1-Krea-dev',
 '/spaces/Qwen/Qwen3-Coder-WebDev',
 '/spaces/Wan-AI/Wan-2.2-5B',
 '/spaces',
 '/datasets/spatialverse/InteriorGS',
 '/datasets/nvidia/Nemotron-Post-Training-Dataset-v1',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/AI-MO/NuminaMath-LEAN',
 '/datasets/UCSC-VLAA/GPT-Image-Edit-1.5M',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer

In [40]:
filtered_links = get_links("https://huggingface.co")

In [41]:
filtered_links['links']

[{'type': 'all-Endpoints', 'url': 'https://endpoints.huggingface.co'},
 {'type': 'about', 'url': 'https://huggingface.co/about'},
 {'type': 'contact', 'url': 'https://apply.workable.com/huggingface/'}]

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [63]:
def get_links(website):
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

def get_all_details(url):
    website = Website(url)
    result = "Landing page:\n"
    result += website.get_contents()
    links = get_links(website)
    print("Found links:", links)
    for link in links["links"]:
        try:
            result += f"\n\n{link['type']}\n"
            result += Website(link["url"]).get_contents()
        except:
            continue
        
    return result

In [54]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'login', 'url': 'https://huggingface.co/login'}, {'type': 'docs', 'url': 'https://github.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Get started with Inference in seconds 🚀
Reachy Mini: The Open Robot for AI Builders
Welcome Cohere on the Hub 🔥
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
openai/gpt-oss-120b
Updated
about 15 hours ago
•
37.8k
•
2.26k
openai/gpt-oss-20b
Updated
about 15 hours ago
•
91.2k
•
1.9k
Qwen/Qwen-Image
Updated
about 7 hours ago
•
21.7k
•
1.1k
tencent/Hunyuan-1.8B-Instruct
Updated
about 6 hours ago
•
943
•
519
black-forest-labs/FLUX.1-Krea-dev
Updat

In [55]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [56]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [57]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about', 'url': 'https://endpoints.huggingface.co'}, {'type': 'docs', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise', 'url': 'https://endpoints.huggingface.co/enterprise'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nGet started with Inference in seconds 🚀\nReachy Mini: The Open Robot for AI Builders\nWelcome Cohere on the Hub 🔥\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nopenai/gpt-oss-120b\nUpdated\nabout 15 hours ago\n•\n37.8k\n•\n2.26k\nopenai/gpt-oss-20b\nUpdated\nabout 15 hours ago\n•\n91.2k\n•\n1.9k\nQwen/Qwen-Image\nUpdated\nabout 7 hours ago\n•\n21.7k\n•\n1.1k\ntencent/Hunyuan-1.8B-Instruct\nUpdated\nabout 7 hours ago\n•\n943\n•\n

In [58]:
def create_brochure(company_name, url):
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [59]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page at https://huggingface.io', 'url': 'https://huggingface.co'}]}


**Hugging Face: The AI Community Building the Future**

### About Us

Hugging Face is a platform where the machine learning community collaborates on models, datasets, and applications. We aim to facilitate the growth of artificial intelligence by providing a comprehensive set of tools and resources.

Our journey began with the development of multiple transformer-based architectures for natural language processing tasks. These frameworks quickly gained popularity among researchers and practitioners worldwide, leading us to expand our offerings beyond just these specific models. Today, we offer:

* **Models:** We have an extensive library of pre-trained models from various domains, including but not limited to natural language understanding, computer vision, and more.
* **Datasets:** An enormous collection of datasets, categorized into various topics such as text classification, object detection, and sentiment analysis, allowing users to test their models on diverse scenarios.
* **Spaces:** Platforms where developers can run experiments with our AI models, sharing results openly to foster collaboration and knowledge-sharing within the community.

### Our Platform

Our platform is built upon a robust API that enables seamless integration with most development environments. With multiple endpoints available, users can easily deploy or fetch their preferred models onto their production workflows.

#### Explore AI Apps

Browse 1M+ applications on our platform, including:

*   **Inference Endpoints:** Deploy our models directly on inference-enabled devices for lightning-fast performance.
*   **Text Generation Inference:** Utilize our text generation capabilities with TGI optimized toolkit to serve language models with unprecedented efficiency.

#### Current Top Models

**Trending Models:**

| Model Name | Model Type (Multi-Protocol) |
| --- | --- |
| nvidia/Nemotron-Post-Training-Dataset-v1 | Transformer |
| openai/gpt-oss-20b | Transformer |
| spatialverse/InteriorGS | Transformers |

### Careers & Jobs

At Hugging Face, we value innovation and collaborate with experts across various disciplines. Explore our job listings below:

*   **Researcher:** Work on enhancing and developing novel AI models, datasets, or applications.
*   **Engineer:** Contribute to the design, development, and optimization of platforms supporting multiple AI techniques.

### Get Started

Join our vibrant community today!

### Sign Up
**Getting started with Inference in seconds 🚀**

 Reachy Mini: The Open Robot for AI Builders

Welcome Cohere on the Hub 🔥

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [69]:
def stream_brochure(company_name, url):
    stream = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [64]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://full.url/endpoints.huggingface.co'}, {'type': 'company page', 'url': 'https://changelog.huggingface.co'}, {'type': 'docs', 'url': 'https://docs.huggingface.co'}]}


# Hugging Face: A Community Building Platform for Artificial Intelligence

## Overview
Hugging Face is an open-source platform that enables the collaboration of machine learning (ML) communities. It provides a unified space to explore AI, build applications, and share knowledge.

## Models

### Open-Source Models

The Hugging Face platform offers a vast collection of pre-trained models in various categories such as:

* Visual: Qwen/Qwen-Image, Tencent/Hunyuan-1.8B-Instruct
* Language: openai/gpt-oss-120b, black-forest-labs/FLUX.1-Krea-dev

These models can be fine-tuned and used to build a wide range of applications.

### Tutorials and Guides

The official documentation provides step-by-step tutorials on how to use the platform, including:

* Generating images from text prompts
* Creating web application code from descriptions
* Training transformers LMs with reinforcement learning

## Datasets

The Hugging Face platform hosts over 1 million models and datasets, including:

* Spatialverse/InteriorGS dataset
* NVIDIA/Nemotron-Post-Training-Dataset-v1 dataset
* AwesomeChatgpt-Prompts dataset

These datasets can be used for any ML task to help improve model performance.

## Spaces

The Hugging Face platform provides several services that enable collaboration and deployment on public models, including:

### DeepSite v2

Generate applications directly with DeepSeek.

### Text Generation Inference

Serve language models with TGI optimized toolkit.

Training multiple GPUs and TPUs in a few clicks is simplified by deploying models on top of the Hugging Face platform.

## Pricing

The cost depends on the compute engine used, ranging from $0.60/hour for GPU to more advanced options available upon request.

By leveraging these resources, developers can accelerate their AI projects with ease.

## Careers/Jobs

Hugging Face is actively hiring engineers, researchers, and business development professionals to work on its diverse range of services. For more information, check the [jobs page](https://huggingface.co/careers).

### Team & Enterprise

Given your enterprise value, sign up for a custom plan to get started.

Single-Sign-On
Regions: Priority Support
Audit Logs: Get real-time records.
Resource Groups: Manage and allocate resources based on user needs.
Private Datasets Viewer: Review datasets before making them public. More than 50,000 organizations are using Hugging Face in enterprise environments.

## About

Hugging Face is building the foundation of ML tooling with a community driven approach. State-of-the-art transformer models have been developed for PyTorch.

### Models
Explore AI Apps
Browse 1M+ models
Text, image, video, audio or even 3D.
Build your portfolio.
Sign Up

### Careers & Jobs
Get started with Inference in seconds: Reachy Mini: The Open Robot for AI Builders
Welcome Cohere on the Hub: The Home of Machine Learning
The collaboration platform for AI experts.

Accelerate your ML with Hugging Face's paid compute and enterprise solutions. Custom plans are available.

### Company

Learn about our company and vision on [about page](#about).

### Docs

Browse the official documentation, including guides, tutorials and more.

[Company Page](#company)
[About Page](#about)
[Documentation Page](#documentation)

### Social

Hugging Face is active on Twitter, LinkedIn, Discord. Stay updated with community news and insights.

[Github Page](https://github.com/huggingface)

In [68]:
system_prompt = "You are an humorous assistant that analyzes the contents of several relevant pages from a company website \
and creates a funny, short brochure about the company for gen-z customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [70]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'enterprise', 'url': 'https://endpoints.huggingface.co'}]}


**Welcome to Hugging Face: The AI Community Building the Future**

As a member of the Hugging Face community, you're part of a vibrant ecosystem where machine learning enthusiasts come together to share knowledge, collaborate on projects, and pursue their passions.

**Our Mission**

Hugging Face is dedicated to building the future of artificial intelligence by providing access to high-quality models, datasets, and tools. Our mission is to empower developers, researchers, and practitioners to create innovative AI solutions that transform industries and improve lives.

** Models (100k+) **

 Explore our vast library of pre-trained models in various categories:
  - **Text Generation**: Serve language models with TGI optimized toolkit
    * Accelerate: Train PyTorch models with multi-GPU, TPU, mixed precision
  - **Image Processing**: Generate images from text prompts using DeepSeek (running on Zero)
  - **Multimodal Processing**: Transformers.js (state-of-the-art ML running directly in your browser)

**Datasets (1M+)**

 Discover and access a wide range of datasets for various machine learning tasks:
  * Spatialverse/InteriorGS
  * Nvidia/Nemotron-Post-Training-Dataset-v1
  * FKA/awesome-chatgpt-prompts

**Spaces (11.5k+)**

 Run your applications on-demand using the DeepSite v2 platform:
  * Generate any application with DeepSeek
  * Running on Zero, generate images from text prompts or web application code from descriptions

**Community**

 Join our vibrant community of developers, researchers, and practitioners to share knowledge, ask questions, and collaborate on projects:

  - Welcome Cohere (official hub for the AI community)
  - Reachy Mini: The Open Robot for AI Builders
  - Qwen/Coder WebDev: Generate web application code from descriptions

**Enterprise**

 Take your machine learning endeavors further with our enterprise-grade solutions:

  - Inference Endpoints by Hugging Face (easily deploy Transformers, Diffusers or any model)
  - Pricing starting at $60/hour for GPU
  - Dedicated support and resources to ensure seamless AI adoption

**Log In / Sign Up**

 Register for a free account to access our platform, models, datasets, and Enterprise solutions:

  * Sign Up | Access Start Here

Stay updated with our latest blog posts, documentation, and events on our website and social media channels:
 - Learn
 - Documentation
 - Blog
 - Forum
 - Press
 - Resources

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>